In [15]:
import numpy as np

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
import inauguralproject

# Question 1

In [24]:
from types import SimpleNamespace

class ExchangeEconomyClass:

    def __init__(self):

        par = self.par = SimpleNamespace()

        # a. preferences
        par.alpha = 1/3
        par.beta = 2/3

        # b. endowments
        par.w1A = 0.8
        par.w2A = 0.3

    def utility_A(self,x1A,x2A):
        util_A = x1A**(self.par.alpha)*x2A**(1-self.par.alpha)
        return util_A
        print(util_A)

    def utility_B(self,x1B,x2B):
        pass

    def demand_A(self,p1):
        pass

    def demand_B(self,p1):
        pass

    def check_market_clearing(self,p1):

        par = self.par

        x1A,x2A = self.demand_A(p1)
        x1B,x2B = self.demand_B(p1)

        eps1 = x1A-par.w1A + x1B-(1-par.w1A)
        eps2 = x2A-par.w2A + x2B-(1-par.w2A)

        return eps1,eps2

In [11]:
from inauguralproject import ExchangeEconomyClass

utilityA = ExchangeEconomyClass()


In [25]:
utilityA = ExchangeEconomyClass()

utilityA.utility_A(1, 0.5)


0.6299605249474366